# KIKA Tutorial: Setting up Sensitivity Analysis with PERT cards

This tutorial demonstrates how to prepare MCNP input files for sensitivity analysis using KIKA. We will:

1. Read an existing MCNP input file
2. Create a perturbed material for a specific nuclide
3. Generate PERT cards for sensitivity calculations
4. Prepare the input for MCNP execution

After completing these steps, you'll have an MCNP input file ready for sensitivity analysis calculations. The results can then be analyzed using the techniques covered in the sensitivity analysis tutorial.

In [1]:
import kika
from pathlib import Path

## 1. Set up the file paths

First, let's define the paths for our example files. In this tutorial, we'll use an example MCNP input file.

In [2]:
# Setup paths
repo_root = Path.cwd().resolve().parent
data_dir = repo_root / 'examples' / 'data'

# Input file path
inputfile = data_dir / 'inputfile_example_0.i'

# Create a working copy of the input file
working_file = data_dir / 'inputfile_example_0_working.i'

with open(inputfile, 'r') as f_in, open(working_file, 'w') as f_out:
    f_out.write(f_in.read())
    print(f"Created working copy: {working_file}")

Created working copy: /home/MONLEON-JUAN/kika/examples/data/inputfile_example_0_working.i


## 2. Read the MCNP input file

Let's read the MCNP input file using the `read_mcnp` function to understand its structure and contents, particularly the material we want to perturb.

In [3]:
# Read the input file
input_data = kika.read_mcnp(working_file)

# Check if our target material exists
material_number = 300000
if material_number in input_data.materials.by_id:
    print(f"Material {material_number} found in the input file")
    
    # Display material composition
    display(input_data.materials.by_id[material_number])

TypeError: Material.add_nuclide() got an unexpected keyword argument 'zaid'

## 3. Create a perturbed material

For sensitivity analysis, we first need to perturb the material of interest. We'll use `perturb_material` to create a version of material 300000 with a 100% increase in Fe-56 content.

The `perturb_material` function takes a `MaterialCollection` as its first argument and modifies it in place (if `in_place=True`). The perturbed material is automatically added to the collection.

In [ ]:
# Read the input file first
input_data = kika.read_mcnp(working_file)

# Perturb Fe-56 of material 300000
# The new API takes the materials collection as first argument
perturbed_mat = kika.perturb_material(
    input_data.materials,   # MaterialCollection (positional arg)
    material_id=300000,
    nuclide='Fe56',         # Can also use ZAID: 26056
    density=-7.85,          # Negative = g/cm³
    pert_mat_id=None,       # Will use default: material_id*100 + 1 = 30000001
    in_place=True,          # Adds perturbed material to the collection
    fraction_type='ao'      # Output as atomic fractions
)

print("Perturbed material created:")
print(perturbed_mat)

Perturbation details:
- Original material: 300000
- Perturbed material ID: 30000001
- Perturbed nuclide: 26056
- Original density: -7.850000e+00 g/cm³ | 8.526728e-02 atoms/barn-cm
- Perturbed density: -1.482784e+01 g/cm³ | 1.603930e-01 atoms/barn-cm

Success! Material written to: /home/MONLEON-JUAN/KIKA/examples/data/inputfile_example_0_working.i


Now we need to write the updated materials back to the input file. The `write_to_mcnp` method writes only new or modified materials:

In [ ]:
# Write the updated materials (including the new perturbed material) to the file
updated_ids = input_data.materials.write_to_mcnp(
    input_filepath=working_file,
    output_filepath=None,  # None = update in place
    force_rewrite=False    # Only update changed/new materials
)

print(f"Updated material IDs: {updated_ids}")

Verify that KIKA has added the perturbed material into the input file.

## 4. Generate PERT cards for sensitivity analysis

Now we'll generate the PERT cards for sensitivity analysis using the `generate_PERTcards` function. We'll specify:
- Cells to apply perturbation (3, 5, 7, 9)
- Reactions of interest: (1, 2, 4, 51, 102)
- Energy grid (SCALE44)
- Order (2 for both first and second order perturbations)

The PERT cards will be appended to the working file. We're using the built-in SCALE44 energy grid from KIKA, but any custom energy grid can also be used. Built-in energy grids are accessible through `kika.energy_grids.[name_of_energy_structure]`.

Note: The density parameter is now optional - if not provided, `generate_PERTcards` will read it from the material definition in the input file.

In [ ]:
# Define cells, reactions and energy grid
cells = [3, 5, 7, 9]  # Cells where perturbation will be applied
reactions = [1, 2, 4, 51, 102]  # MT numbers of reactions
perturbed_material_id = 300000 * 100 + 1  # Default: material_id * 100 + 1 = 30000001

# We'll use SCALE44 energy grid from KIKA
print(f"Energy grid has {len(kika.energy_grids.SCALE44)} points, creating {len(kika.energy_grids.SCALE44)-1} energy bins")

# Generate PERT cards
# The density is now read from the material definition in the input file
kika.generate_PERTcards(
    inputfile=working_file,
    cell=cells,
    reactions=reactions,
    material=perturbed_material_id,
    energies=kika.energy_grids.SCALE44,
    # density is optional - will be read from material if not provided
    order=2,  # Generate both first and second order perturbations
    errors=False,  # Don't generate error method cards
    in_place=True  # Add PERT cards to the input file
)

Energy grid has 45 points, creating 44 energy bins

Success! PERT cards written to: /home/MONLEON-JUAN/KIKA/examples/data/inputfile_example_0_working.i


Verify that the KIKA has added the PERT cards into the input file.

## 5. Examining the generated PERT cards

Let's take a look at the first few PERT cards that were generated to understand their format.

In [9]:
# Read back the file to examine the PERT cards
modified_input_data = kika.read_mcnp(working_file)

# Display the PERT cards
display(modified_input_data.perturbation)

                   MCNP Perturbation Data                   

Number of perturbations:  440
Perturbation numbers:     1-440
Particle types:           n
Reactions available:      1, 2, 4, 51, 102
Methods available:        2, 3
Energy range:             1.00e-11 - 2.00e+01 MeV
Number of energy bins:    44
Energy structure:         SCALE44


Examples of accessing data:
- .pert[perturbation_number] - Access a specific perturbation

## 6. Running MCNP with the prepared input

Now that we have prepared the input file with perturbed material and PERT cards, it's ready for MCNP execution.

## 7. Next Steps: Analyzing Sensitivity Results

After running MCNP with the prepared input file, you'll have output files including a MCTAL file which contains the perturbation results.

For analyzing these results and calculating sensitivity coefficients, refer to the companion tutorial notebook `tutorial_sensitivity.ipynb`, where you'll learn how to:

1. Compute sensitivity coefficients from MCNP input and output files
2. Explore sensitivity data structures
3. Visualize sensitivity profiles
4. Analyze nonlinearity with Taylor series expansion
5. Export sensitivity data to SDF format

The typical workflow would be:
```python
# Compute sensitivity coefficients for Fe-56
sens_fe56 = kika.compute_sensitivity(
    inputfile="inputfile_example_0_working.i",
    mctalfile="inputfile_example_0.m", 
    tally=4,  # Tally number to analyze
    zaid=26056,  # Fe-56
    label="Fe-56 Sensitivity"
)

# Visualize sensitivity profiles
sens_fe56.plot_sensitivity(
    energy="integral",
    reaction=[1, 2, 4, 51, 102],
    xlim=(0, 10)  # Limit x-axis to 0-10 MeV
)
```

## 8. Summary

In this tutorial, we've learned how to prepare an MCNP input file for sensitivity analysis:

1. **Reading MCNP input files** using `read_mcnp`
2. **Perturbing materials** using `perturb_material` to create a 100% perturbation for Fe-56
3. **Generating PERT cards** using `generate_PERTcards` with specific cells, reactions, and energy grid
4. **Preparing for MCNP execution** and understanding the next steps for analysis

This workflow allows you to systematically evaluate the sensitivity of your model to various nuclear data parameters, which is essential for uncertainty quantification and validation studies.